<a href="https://colab.research.google.com/github/sallyelbaz/Automated-Multi-Source-Data-Pipeline-/blob/main/Multi_Source_Data_Pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Automated Multi-Source Data Pipeline

Problem: Data is fragmented across many internal systems. Analysts manually pull data, causing
delays and errors.

Solution: Build an automated pipeline that connects to multiple data sources (APIs, SQL database, flat
files), cleans the data, and loads it into a cloud warehouse (BigQuery/Snowflake). Set up scheduled
automation.


Tech Stack: Python, SQL, Airflow, dbt, BigQuery/Snowflake, GitHub Actions
Expected Business Impact: Reduced manual reporting effort by 90%. Enabled real time
decision making.

In [68]:
!pip install --upgrade google-cloud-bigquery
!pip install --upgrade pandas-gbq

In [70]:
# loading the necessary libraries
import pandas as pd
import numpy as np
from google.cloud import bigquery
from google.oauth2 import service_account
from google.colab import auth

In [72]:
# Authenticating
auth.authenticate_user()

In [11]:
# loading the customer data
cust_url = "https://raw.githubusercontent.com/sallyelbaz/Automated-Multi-Source-Data-Pipeline-/refs/heads/main/data/customers.csv"
cust = pd.read_csv(cust_url)
cust.head()

,customer_id,first_name,last_name,country,signup_date
0,1,Liam,Smith,Germany,2023-01-01
1,2,Olivia,Garcia,France,2023-01-02
2,3,Sophia,Garcia,UK,2023-01-03
3,4,Olivia,Brown,US,2023-01-04
4,5,Olivia,Johnson,UK,2023-01-05


In [64]:
# Standardizing Customer Data
# renaming the columns
cust = cust.rename(columns={
    "ID": "CustID",
    "first_name": "First Name",
    "last_name": "Last Name",
    "country":"Country",
    "signup_date" : "Sign Up Date"
})
# Converting data types
cust = cust.astype({
    "First Name": "str",
    "Last Name" : "str",
    "Country" : "str",
    "Sign Up Date":"datetime64[ns]"
})

,CustID,First Name,Last Name,Country,Sign Up Date
0,1,Liam,Smith,Germany,2023-01-01
1,2,Olivia,Garcia,France,2023-01-02
2,3,Sophia,Garcia,UK,2023-01-03
3,4,Olivia,Brown,US,2023-01-04
4,5,Olivia,Johnson,UK,2023-01-05


In [6]:
# loading the orders data
orders_url = "https://raw.githubusercontent.com/sallyelbaz/Automated-Multi-Source-Data-Pipeline-/refs/heads/main/data/orders.csv"
orders = pd.read_csv(orders_url)
orders.head()

,order_id,customer_id,order_amount,order_date
0,1001,324,185.03,2024-01-01
1,1002,146,111.51,2024-01-02
2,1003,446,400.11,2024-01-03
3,1004,293,337.89,2024-01-04
4,1005,153,261.64,2024-01-05


In [66]:
# Standardizing Order Data
# Changing Column Names
orders = orders.rename(columns={
  "ID": "OrderID",
  "customer_id": "CustID",
  "order_amount": "Order Amount",
  "order_date" : "Order Date"
})
# Changing Data Types
orders = orders.astype({
    "Order Date":"datetime64[ns]"
})
print(orders.dtypes)

OrderID                  int64
CustID                   int64
Order Amount           float64
Order Date      datetime64[ns]
dtype: object


In [67]:
# Merging the data together into one table
cust_orders = pd.merge(cust,orders,on="CustID")
cust_orders.head()

,CustID,First Name,Last Name,Country,Sign Up Date,OrderID,Order Amount,Order Date
0,2,Olivia,Garcia,France,2023-01-02,1422,34.12,2025-02-25
1,3,Sophia,Garcia,UK,2023-01-03,1078,42.42,2024-03-18
2,3,Sophia,Garcia,UK,2023-01-03,1482,339.15,2025-04-26
3,4,Olivia,Brown,US,2023-01-04,1277,178.60,2024-10-03
4,5,Olivia,Johnson,UK,2023-01-05,1058,485.47,2024-02-27


In [73]:
# Setting the BigQuery Parameters
project_id = "prime-hour-477815-g3"
dataset_id = "CustOrders"
table_id = "CustOrders"
destination_table = f"{project_id}.{dataset_id}.{table_id}"

cust_orders.to_gbq(
    destination_table=destination_table,
    project_id=project_id,
    if_exists='replace'   # replace existing table
)

/tmp/ipython-input-920111788.py:7: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  cust_orders.to_gbq(
100%|██████████| 1/1 [00:00<00:00, 5295.84it/s]
